In [1]:
!pip install xmltodict


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from requests import Session

import threading
import random
import time

import utils as ut
import html_utils as hu

try:
    ut.fu.copy_folder_structure(
        ut.song_ids_of_song_parts_dir,
        ut.song_parts_from_ids_dir
    )
except Exception as e:
    print(e)

start_letter = 'a'
end_letter = 'z'

song_parts_list = []

error_song_parts_list = []

In [3]:
lock = threading.Lock()

thread_per_batch = 20

thread_list = []

total_saved = 0

In [4]:
class Downloader():
    host = "https://api.hooktheory.com/v1/songs/public/"
    song_parts_item = None
    client = None

    json_file_path = None
    song_link = None

    def __init__(self):
        self.client = Session()

        time_to_sleep = random.uniform(0, 1)
        time.sleep(time_to_sleep)
        
        with lock:
            if len(song_parts_list) > 0:
                self.song_parts_item = song_parts_list.pop(0)
            else:
                self.song_parts_item = None

        if self.song_parts_item is not None:
            self.song_link = self.song_parts_item["link"]
            self.json_file_path = ut.fu.song_link_to_relative_html_file_path(self.song_link)
            self.json_file_path = self.json_file_path.replace(".html", ".json")

            self.json_file_path = ut.fu.relative_to_absolute_path(
                relative_path = self.json_file_path,
                root_path = ut.song_parts_from_ids_dir
            )
        else:
            pass

    def download(self):
        song_part_ids = list(self.song_parts_item["song_parts_ids"])

        song_entire_data = dict(self.song_parts_item)

        song_entire_data["song_parts"] = []
        del song_entire_data["song_parts_ids"]

        for idx in range(len(song_part_ids)):
            song_part_id = song_part_ids[idx]
            print(f"--------> PART {idx} OF {self.song_parts_item['link']}")

            song_part_url = f"{self.host}{song_part_id}"
            response = hu.make_request(self.client, song_part_url)

            if response["status_code"] == 200:
                if response["main_data"] is not None:
                    song_entire_data["song_parts"].append(
                        {
                            "song_part_id": song_part_id,
                            "metadata": response["metadata"],
                            "main_data_type": response["main_data_type"],
                            "main_data": response["main_data"]
                        }
                    )

                    time.sleep(random.uniform(2, 5))
                else:
                    error_message = f"FAIL: download - empty main_data: {song_part_url} -> {response}"
                    print(error_message)
                    print()
                    break
            else:
                error_message = f"FAIL: download - status_code: {response['status_code']}: {ut.http_status_code[response['status_code']]}, url: {song_part_url}"
                print(error_message)
                print()
                break

        n_part_downloaded = len(song_entire_data["song_parts"])
        n_part_total = len(song_part_ids)

        if n_part_downloaded == n_part_total:
            ut.fu.write_data_to_json_file(
                song_entire_data,
                self.json_file_path
            )
            
            global total_saved

            with lock:
                total_saved += 1
                print(f"--------> TOTAL SAVED: {total_saved}")
                print

            print(f"--------> SAVED: {self.json_file_path}")
            print(f"SUCCESS: {self.song_link}")
            print()
        else:
            print(f"ACK: {n_part_downloaded}/{n_part_total} part(s) downloaded - {self.song_link}")

            with lock:
                error_song_parts_list.append(self.song_parts_item)
            
            print(f"--------> APPENDED to error_song_parts_list: {self.song_link}")
            print()

    def download_song_parts(self):
        if self.song_parts_item is not None:
            print(f"GETTING: {self.song_parts_item['link']}")
            time.sleep(0.5)
            self.download()
        else:
            print("WARNING: download_song_parts - No more song to handle")
            print()

In [5]:
for letter_to_crawl in range(ord(start_letter), ord(end_letter) + 1):
    song_parts_list.extend(
        ut.read_song_id_of_song_parts_list(
            chr(letter_to_crawl),
            min_json_result_file_size=1000
        )
    )

    while len(song_parts_list) > 0:
        for i in range(thread_per_batch):
            downloader = Downloader()
            thread = threading.Thread(target=downloader.download_song_parts)
            thread_list.append(thread)
            thread.start()

        for thread in thread_list:
            thread.join()

        thread_list.clear()

    print()
    print(f"--------> LETTER {chr(letter_to_crawl)} DONE")
    print()

EXIST: D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/DataPreprocessing/song_parts_from_ids/a/arthur/beginning-of-time.json
{'artist_name': 'Arthur', 'artist_link': 'https://www.hooktheory.com/theorytab/artists/a/arthur', 'song_name': 'Beginning of time', 'link': 'https://www.hooktheory.com/theorytab/view/arthur/beginning-of-time', 'metrics': {}, 'metrics_conclusion': '', 'key_conclusion': '', 'song_parts': [{'song_part_id': 'nLgaejbdgYp', 'metadata': None, 'main_data_type': None, 'main_data': None}]}
GETTING: https://www.hooktheory.com/theorytab/view/arthur/beginning-of-time

--------> PART 0 OF https://www.hooktheory.com/theorytab/view/arthur/beginning-of-time
--------> REQUESTING: https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp

-----------> FAIL (trying other parser): make_request - exception: no element found: line 1, column 0 - https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp
-----------> TRYING OTHER PARSER: https://api.hooktheory.com/v1/songs/publ

In [6]:
print(error_song_parts_list)
print(len(error_song_parts_list))
raise Exception("STOP")

[{'artist_name': 'Arthur', 'artist_link': 'https://www.hooktheory.com/theorytab/artists/a/arthur', 'song_name': 'Beginning of time', 'link': 'https://www.hooktheory.com/theorytab/view/arthur/beginning-of-time', 'metrics': {}, 'metrics_conclusion': '', 'key_conclusion': '', 'song_parts_ids': ['nLgaejbdgYp']}, {'artist_name': 'Katy Perry', 'artist_link': 'https://www.hooktheory.com/theorytab/artists/k/katy-perry', 'song_name': 'Teenage Dream', 'link': 'https://www.hooktheory.com/theorytab/view/katy-perry/teenage-dream', 'metrics': {'Chord Complexity': '60', 'Melodic Complexity': '64', 'Chord-Melody Tension': '53', 'Chord Progression Novelty': '71', 'Chord Bass Melody': '79'}, 'metrics_conclusion': ' In terms of chords and melody, Teenage Dream is significantly more complex than the typical song, having above average scores in Chord Complexity, Melodic Complexity, Chord-Melody Tension, Chord Progression Novelty and Chord-Bass Melody. ', 'key_conclusion': ' Teenage Dream is written in the 

Exception: STOP

In [ ]:
song_parts_list = error_song_parts_list

while len(song_parts_list) > 0:
    for i in range(thread_per_batch):
        downloader = Downloader()
        thread = threading.Thread(target=downloader.download_song_parts)
        thread_list.append(thread)
        thread.start()

    for thread in thread_list:
        thread.join()

    thread_list.clear()

GETTING: https://www.hooktheory.com/theorytab/view/arthur/beginning-of-time
GETTING: https://www.hooktheory.com/theorytab/view/katy-perry/teenage-dream
GETTING: https://www.hooktheory.com/theorytab/view/katy-perry/et
--------> PART 0 OF https://www.hooktheory.com/theorytab/view/arthur/beginning-of-time
--------> REQUESTING: https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp
--------> PART 0 OF https://www.hooktheory.com/theorytab/view/katy-perry/teenage-dream
--------> REQUESTING: https://api.hooktheory.com/v1/songs/public/dPoDEWjZmnM
--------> PART 0 OF https://www.hooktheory.com/theorytab/view/katy-perry/et
--------> REQUESTING: https://api.hooktheory.com/v1/songs/public/RzojNvRwoDe

-----------> FAIL (trying other parser): make_request - exception: no element found: line 1, column 0 - https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp
-----------> TRYING OTHER PARSER: https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp
-----------> FAIL: make_request - exception: no eleme

KeyboardInterrupt: 

--------> PART 0 OF https://www.hooktheory.com/theorytab/view/arthur/beginning-of-time
--------> REQUESTING: https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp
-----------> FAIL (trying other parser): make_request - exception: no element found: line 1, column 0 - https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp
-----------> TRYING OTHER PARSER: https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp
-----------> FAIL: make_request - exception: no element found: line 1, column 0 - https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp
-----------> WARNING: make_request - empty main_data: https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp
FAIL: download - empty main_data: https://api.hooktheory.com/v1/songs/public/nLgaejbdgYp -> {'status_code': 200, 'metadata': None, 'main_data_type': None, 'main_data': None}

ACK: 0/1 part(s) downloaded - https://www.hooktheory.com/theorytab/view/arthur/beginning-of-time
--------> APPENDED to error_song_parts_list: https://www.hooktheory.co